In [48]:
import pandas as pd

annotator_names = ['Oscar', 'Forsberg']
annotations = []

for name in annotator_names:
    annotations.append(pd.read_excel('annotations/LLM Prediction Annotations(1).xlsx', sheet_name=name))

for annotation in annotations:
    annotation['Annotation'] = annotation['LLM 1 Description'].fillna('No description')
    print(annotation.head(3))

KeyError: 'LLM 1 Description'

In [44]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

# Agreememt percentage
agreement = np.zeros((len(annotations), len(annotations)))
cohen_kappa = np.zeros((len(annotations), len(annotations)))

for anno_idx in range(len(annotations)):
    for i in range(len(annotations[anno_idx])):
        for anno_2_idx in range(len(annotations)):
            agreement[anno_idx, anno_2_idx] = np.mean(annotations[anno_idx]['Annotation'] == annotations[anno_2_idx]['Annotation'])
            cohen_kappa[anno_idx, anno_2_idx] = cohen_kappa_score(annotations[anno_idx]['Annotation'], annotations[anno_2_idx]['Annotation'])

agreement_df = pd.DataFrame(agreement, columns=annotator_names, index=annotator_names)
cohen_kappa_df = pd.DataFrame(cohen_kappa, columns=annotator_names, index=annotator_names)

print(f"The total number of annotations: {len(annotations[0])}\n")
print(f"The agreement percentage between the annotators:\n{agreement_df}\n")
print(f"The Cohen's Kappa between the annotators:\n{cohen_kappa_df}") 
# Kappa value interpretation Landis & Koch (1977):
# <0 No agreement
# 0 — .20 Slight
# .21 — .40 Fair
# .41 — .60 Moderate
# .61 — .80 Substantial
# .81–1.0 Perfect


The total number of annotations: 798

The agreement percentage between the annotators:
           Ture     Oscar
Ture   1.000000  0.528822
Oscar  0.528822  1.000000

The Cohen's Kappa between the annotators:
           Ture     Oscar
Ture   1.000000  0.312132
Oscar  0.312132  1.000000
